In [1]:
# lasso regression
# https://www.kaggle.com/mymkyt/simple-lasso-public-score-0-12102/output
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import skew
from sklearn.metrics import r2_score

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')


all_data = pd.concat((train.loc[:,'MSSubClass':'SaleCondition'],
                      test.loc[:,'MSSubClass':'SaleCondition']))

train["SalePrice"] = np.log1p(train["SalePrice"])

numeric_feats = all_data.dtypes[all_data.dtypes != "object"].index

skewed_feats = train[numeric_feats].apply(lambda x: skew(x.dropna())) #compute skewness
skewed_feats = skewed_feats[skewed_feats > 0.75]
skewed_feats = skewed_feats.index

all_data[skewed_feats] = np.log1p(all_data[skewed_feats])

all_data = pd.get_dummies(all_data)

all_data = all_data.fillna(all_data.mean())

X_train = all_data[:train.shape[0]]
X_test = all_data[train.shape[0]:]
y_train = train.SalePrice
y_test = test.SalePrice


#### models selection

from sklearn.linear_model import Lasso

lasso = Lasso(alpha=0.0004)

model = lasso

### prediction
model.fit(X_train, y_train)

preds = np.expm1(model.predict(X_test))
solution = pd.DataFrame({"id":test.Id, "SalePrice":preds})
solution.to_csv("lasso.csv", index = False)

print ('done')
print (preds)

r2_score_lasso = r2_score(y_test, preds)
print ('R2=', r2_score_lasso)

done
[120413.70856287 151628.2011919  180433.10137552 ... 169027.50464335
 118467.80077457 226524.27189441]
R2= -18.4147632437051
